In [27]:
import pandas as pd
import numpy as np
import os, re, time
import sqlite3 as lite
from nyc_geoclient import Geoclient

In [28]:
# read-in NYC Geoclient API token
token=open('NYC_Geoclient_token.txt')
g=Geoclient(token.readline().strip('\n'), token.readline().strip('\n'))

In [29]:
def clean_strings(x):
    try:
        return str(x).strip()
    except ValueError:
        return np.nan
    
def parse_address(address):
    separators=['Apt','APT','#']
    # gets rid of the apartments in the address
    for separator in separators:
        if address.find(separator)!=-1:
            no_apt=address.split(separator,1)[0]
        else:
            no_apt=address
        #apartments can also be indicated by comma followed by number with optional letter (ex: , 503C)
        street=re.split(r'(,\s\d+$|,\s\d+\w{1}$)', no_apt)
        street=street[0] # grab what came before the apartment
        
        #separate address into street number and street name.
    split=re.split(r'(^\d+\s|^\d+-\d+\s|^\d+\w{1}\s|^\d+-\d+\w{1}\s)', street)
    if split[0]=='':            
        split.remove(split[0])
#     print split
    # strip trailing spaces
    return [_.strip() for _ in split]
        
def AddressMatch(str_num,str_name,borough): ## function to geocode street addresses
    geocode=g.address(str_num, str_name, borough)
    message=geocode.get('message')
    latitude=geocode.get('latitude')
    longtitude=geocode.get('longitude')
    GeocodeResult='Address Match'
    return [longtitude, latitude, GeocodeResult, message]

def BlockMatch(borough, block,lot): ## function to geocode blocks and lots
    geocode=g.bbl(borough, block,lot)
    message=geocode.get('message')
    latitude=geocode.get('latitudeInternalLabel')
    longtitude=geocode.get('longitudeInternalLabel')
    GeocodeResult='Block Match'
    return [longtitude, latitude, GeocodeResult, message]

def Geocode(df):
    start=int(input('''From what line should I start? 
    Type in numercial value; type 0 (zero) for the first iteration \n'''))
    counter=0
    for index, row in df.iterrows():
        # geocode only from starting passed as input
        if index==0 or index > start:    
            counter=counter+1
            if counter % 5000==0:
                print '{} records have been geocoded'.format(counter)
            if counter%50==0:
                # will pause for 1 second after 50 geocoded records
                time.sleep(1)
            try:
                # do Address match first
                result=AddressMatch(row['street_number'], row['street_name'], int(row['borough']) )

                # if longtitude is None-->invalid addres, then try BlockMatch function
                if result[0] is None:
                    result=BlockMatch(int(row['borough']), int(row['block']), int(row['lot']) )

                    # if BlockMatch didn't return longtitude, mark the record as Unmatched in place of result
                    if result[0] is None:
                        result[2]='Unmatched'

                # for the db, remove last two items (parsed addrres) from the dataframe row and add geocoded results
                db_row=list(row[0:-2])+result

                # writing geocoded record into a database
                cur=con.cursor()
                table_name='yr{}'.format(year)
                cur.execute('''CREATE TABLE IF NOT EXISTS %s (sale_id INTEGER PRIMARY KEY, bbl_id INTEGER, 
                year TEXT, borough INTEGER,nbhd TEXT, bldg_ctgy TEXT,
                tax_cls_p TEXT, block TEXT,lot TEXT,easmnt TEXT, bldg_cls_p TEXT,address TEXT,
                apt TEXT, zip TEXT, res_unit INTEGER,com_unit INTEGER, tot_unit INTEGER, land_sqft INTEGER,
                tot_sqft INTEGER, yr_built INTEGER, tax_cls_s TEXT, bldg_cls_s TEXT,sale_date TEXT, price INTEGER,
                usable TEXT, long REAL, lat REAL, georesult TEXT, message TEXT)'''% table_name)

                qMark='?,'*28
                placeholder=qMark[:-1]       
                cur.execute('''INSERT INTO %s(bbl_id, year, borough, nbhd, bldg_ctgy, tax_cls_p, block,lot,
                easmnt, bldg_cls_p, address, apt, zip, res_unit, com_unit, tot_unit, land_sqft, tot_sqft,
                yr_built, tax_cls_s, bldg_cls_s, sale_date, price, usable, long, lat, 
                georesult, message) VALUES (%s)''' % (table_name,placeholder), db_row)        
                con.commit()    

            except Exception as e:
                print e
                print('An error has occurred. File stopped at index {}'.format(index))
                break
    con.close()    
    print 'Done'

In [30]:
data_path='/Users/anastasiaclark/NYC_RE_Sales'
# turn year into user input after testing
year='2017'

In [31]:
# read each borough sales and put it in a list temporarily
df_list=[]
data_folder=os.path.join(data_path,year)
boro_sales=[table for table in os.listdir(data_folder) if not 'citywide_sales' in table ]
for boro_table in boro_sales:
    df=pd.read_excel(os.path.join(data_folder,boro_table),skiprows=[0,1,2,3], parse_dates=True)
    df_list.append(df)

In [32]:
# get all boroughs sales into a single table
sales=pd.concat(df_list, ignore_index=True)
# clean-up column names
sales.columns=[c.strip() for c in sales.columns]

# give shorter names
sales.rename(columns={'BOROUGH': 'borough','NEIGHBORHOOD':'nbhd','BUILDING CLASS CATEGORY':'bldg_ctgy',
                   'TAX CLASS AT PRESENT':'tax_cls_p','BLOCK':'block','LOT':'lot',
                      'EASE-MENT':'easmnt','BUILDING CLASS AT PRESENT':'bldg_cls_p','ADDRESS':'address',
                   'APARTMENT NUMBER':'apt','ZIP CODE':'zip','RESIDENTIAL UNITS':'res_unit',
                   'COMMERCIAL UNITS':'com_unit','TOTAL UNITS':'tot_unit',
                   'LAND SQUARE FEET':'land_sqft','GROSS SQUARE FEET':'tot_sqft',
                   'YEAR BUILT':'yr_built','TAX CLASS AT TIME OF SALE':'tax_cls_s',
                   'BUILDING CLASS AT TIME OF SALE':'bldg_cls_s',
                      'SALE PRICE':'price','SALE DATE':'sale_date'}, 
          inplace=True)

# in 2017 DOF changed the column names, assuming that column 'BUILDING CLASS AS OF FINAL ROLL 17/18' will have different
# ending, locate them using the regex and rename them
sales.rename(columns={sales.filter(regex='BUILDING CLASS AS OF FINAL ROLL*').columns[0]: 'bldg_cls_p', 
                      sales.filter(regex='TAX CLASS AS OF FINAL ROLL*').columns[0]: 'tax_cls_p'}, inplace=True)


In [33]:
# strip trailing spaces from text columns
text_cols=[c for c in sales.columns if sales[c].dtype=='object']
for c in text_cols:
    sales[c]=sales[c].apply(lambda x: clean_strings(x))

# add some columns    
sales['bbl_id']=sales['borough'].astype(str)+sales['block'].astype(str)+sales['lot'].astype(str)
sales['usable']=np.where(sales['price']>10,'True','False')
sales['year']='{}'.format(year)
# SQLite doesn't support pandas datetime format, change dates to text
sales['sale_date']=sales['sale_date'].astype(str)


# re-arrange the order of the columns to be same as in the past
cols_order=['bbl_id', 'year', 'borough', 'nbhd', 'bldg_ctgy', 'tax_cls_p', 'block', 'lot',
            'easmnt', 'bldg_cls_p', 'address', 'apt', 'zip', 'res_unit', 'com_unit', 
            'tot_unit', 'land_sqft', 'tot_sqft', 'yr_built', 'tax_cls_s', 'bldg_cls_s', 
            'sale_date', 'price', 'usable']

sales=sales.loc[:, cols_order]

In [34]:
# separate address into individual parts (needed for geocoding parameters)
sales[['street_number', 'street_name']]=sales.apply(lambda row: pd.Series(parse_address(row['address'])), axis=1)

In [35]:
sales.shape

(88660, 26)

# Testing

In [36]:
sales.index.is_unique

True

In [41]:
# this line runs...
con=lite.connect('RE_Sales_beta.sqlite')
r=Geocode(sales)

From what line should I start? 
    Type in numercial value; type 0 (zero) for the first iteration 
56364
5000 records have been geocoded
10000 records have been geocoded
15000 records have been geocoded
20000 records have been geocoded
25000 records have been geocoded
30000 records have been geocoded
Done


In [ ]:
#AddressMatch(sales.iloc[12503]['street_number'], sales.iloc[12503]['street_name'], sales.iloc[12503]['borough'])

In [79]:
q='''select * from yr2017'''

In [80]:
con=lite.connect('RE_Sales_beta.sqlite')
df=pd.read_sql_query(q,con)

In [81]:
df.columns

Index([u'sale_id', u'bbl_id', u'year', u'borough', u'nbhd', u'bldg_ctgy',
       u'tax_cls_p', u'block', u'lot', u'easmnt', u'bldg_cls_p', u'address',
       u'apt', u'zip', u'res_unit', u'com_unit', u'tot_unit', u'land_sqft',
       u'tot_sqft', u'yr_built', u'tax_cls_s', u'bldg_cls_s', u'sale_date',
       u'price', u'usable', u'long', u'lat', u'georesult', u'message'],
      dtype='object')

In [82]:
# 48 Unmatched rows
df.loc[df['georesult']=='Unmatched']

sale_id     bbl_id  year  borough                       nbhd  \
1905      1906    2294641  2017        2               EAST TREMONT   
3818      3819   22598183  2017        2     MOTT HAVEN/PORT MORRIS   
3884      3885   22850160  2017        2      MOUNT HOPE/MOUNT EDEN   
4849      4850   25859285  2017        2                  RIVERDALE   
6105      6106   23436350  2017        2                  SOUNDVIEW   
7528      7529    2468376  2017        2             WILLIAMSBRIDGE   
13841    13842  122159079  2017        1                     INWOOD   
14261    14262   19359049  2017        1                   KIPS BAY   
14262    14263   19359050  2017        1                   KIPS BAY   
15622    15623  113609002  2017        1               MIDTOWN EAST   
15623    15624  113609002  2017        1               MIDTOWN EAST   
18105    18106   18589053  2017        1                MURRAY HILL   
18106    18107   18589056  2017        1                MURRAY HILL   
27369    27370  516681081  2017        5                 BULLS HEAD   
33121    33122     566551  2017        5                PRINCES BAY   
33123    33124     566621  2017        5                PRINCES BAY   
33767    33768   57577200  2017        5  ROSSVILLE-RICHMOND VALLEY   
34901    34902    5264545  2017        5                     TRAVIS   
34902    34903    5264545  2017        5                     TRAVIS   
35208    35209     514853  2017        5          WEST NEW BRIGHTON   
37975    37976    3178652  2017        3         BEDFORD STUYVESANT   
39427    39428    3384172  2017        3                BOERUM HILL   
40840    40841    3534646  2017        3               BOROUGH PARK   
44313    44314    3290131  2017        3                COBBLE HILL   
47632    47633    3428446  2017        3              EAST NEW YORK   
47828    47829    3436840  2017        3              EAST NEW YORK   
54142    54143    3146858  2017        3                 OCEAN HILL   
54599    54600    3651521  2017        3        OCEAN PARKWAY-NORTH   
54841    54842    3653642  2017        3        OCEAN PARKWAY-NORTH   
54842    54843    3653643  2017        3        OCEAN PARKWAY-NORTH   
57988    57989    3746013  2017        3             SHEEPSHEAD BAY   
58226    58227   34586203  2017        3               SPRING CREEK   
60424    60425  324579039  2017        3         WILLIAMSBURG-SOUTH   
60970    60971   41606425  2017        4                    ARVERNE   
61830    61831      48901  2017        4                    ASTORIA   
61831    61832      48901  2017        4                    ASTORIA   
61832    61833      48901  2017        4                    ASTORIA   
66054    66055    4170058  2017        4              EAST ELMHURST   
68816    68817   45251183  2017        4             FLUSHING-NORTH   
75166    75167    4139875  2017        4               HOWARD BEACH   
75167    75168    4139875  2017        4               HOWARD BEACH   
85794    85795   41174644  2017        4           SOUTH OZONE PARK   
85795    85796   41174644  2017        4           SOUTH OZONE PARK   

                            bldg_ctgy tax_cls_p  block   lot easmnt  \
1905       05 TAX CLASS 1 VACANT LAND        1B   2946    41          
3818        31 COMMERCIAL VACANT LAND         4   2598   183          
3884       05 TAX CLASS 1 VACANT LAND        1B   2850   160          
4849       05 TAX CLASS 1 VACANT LAND        1B   5859   285          
6105       05 TAX CLASS 1 VACANT LAND        1B   3436   350          
7528       05 TAX CLASS 1 VACANT LAND        1B   4683    76          
13841       31 COMMERCIAL VACANT LAND         4   2215  9079          
14261       31 COMMERCIAL VACANT LAND         4    935  9049          
14262       31 COMMERCIAL VACANT LAND         4    935  9050          
15622  10 COOPS - ELEVATOR APARTMENTS         2   1360  9002          
15623  10 COOPS - ELEVATOR APARTMENTS         2   1360  9002          
18105       31 COMMERCIAL VACA